### Gene deletions 

The following script is used to further fit the already created models with gene deletions to further increase the similarity to the *e. coli* metabolic pathways described in the work of Huang, Zhongshi, et al.. 
First, we will see if these knockouts alter the growth rate and second, we will knock out the reactions that are linked to the deleted genes in our 3 different pathways.



In [21]:
# This is just importing of the model and some tools to work with the GSM.
import numpy as np
from cobra.io import read_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
import pandas as pd
from typing import Union
# Read model (central metabolism model of Escherichia coli)
model = read_sbml_model("iML1515.xml")
%run -i create_pathway.ipynb #runs script, has 

In [34]:
# creating the models with the respective pathways
nam_model = create_pathway(model.copy(),1)
na_model = create_pathway(model.copy(),2)
nr_model = create_pathway(model.copy(),3)
models = [(nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]

In [35]:
import pandas
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

### NAM Model: 
For the NAM Model it is not necessary to Knockout the purR gene, because it is not present in our GSM 

### NA Model: 
For the NA Model it is necessary to knock out the pncC gene. The pncC is known as the gene: b2700 in the iML1515 model and is associated to the reaction NMNDA 

In [36]:
print('complete model: ', na_model.optimize())
with na_model:
    na_model.genes.b2700.knock_out()
    print('pncC knocked out: ', na_model.genes.b2700.knock_out())
    

complete model:  <Solution 0.877 at 0x7ff72a139bb0>
pncC knocked out:  None


### NR Model: 
For the NR Model it is important to knock out the nadR and ushA genes, according to the paper by Huang. The UshA gene is known as the b0480 gene in the iML1515 model and is associated to many different reactions. The nadR gene is known as the b4390 gene in our GSM and associated to the NMNAT reaction.  After doing those knock outs we actually found out, that the genes ushA and nadR are as described present in the GSM but catalyse not the reaction, that were described by Huang. 

In [40]:
print('complete model: ', nr_model.optimize())
with nr_model:
    nr_model.genes.b0480.knock_out()
    print('ushA knocked out: ', nr_model.genes.b0480.knock_out())
    nr_model.genes.b4390.knock_out()
    print('nadR knocked out: ', nr_model.genes.b4390.knock_out())
          

complete model:  <Solution 0.877 at 0x7ff72a139c10>
ushA knocked out:  None
nadR knocked out:  None


In the next step we would like to change our models according to the gene deletions. 

### NA Pathway: deleting gene b2700 (PncC) from the NA Model 

In [41]:
def remove_genes(
    model:"na_model",
    gene_list: Union[List["b2700"],Set["Gene"],List[str],Union[str]],
    remove_reactions: bool = True,
) -> None:
    
    rxn_set = set()
    for gene in model.genes.get_by_any(gene_list):
        gene.knock_out()
        rxn_set.update(gene.reactions)
    return [rxn for rxn in rxn_set if not rxn.functional]


NameError: name 'List' is not defined

### NAR Pathway: deleting gene b4390 (NadR)and the gene b0480 (UshA) from the NR Model 

In [ ]:
from cobra.util import get_context
def delete_model_genes(
    model: "nr_model",
    gene_list: Union[List[ "b4390" ], Set["Gene"], List[str], Set[str]],
    cumulative_deletions: bool = True,
    disable_orphans: bool = 
    False,
) ->None:
    
    rxn_set = set()
    for gene in model.genes.get_by_any(gene_list):
        gene.knock_out()
        rxn_set.update(gene.reactions)
    return [rxn for rxn in rxn_set if not rxn.functional]

In [ ]:
from cobra.util import get_context
def delete_model_genes(
    model: "nr_model",
    gene_list: Union[List[ "b0480" ], Set["Gene"], List[str], Set[str]],
    cumulative_deletions: bool = True,
    disable_orphans: bool = 
    False,
) ->None:
    
    rxn_set = set()
    for gene in model.genes.get_by_any(gene_list):
        gene.knock_out()
        rxn_set.update(gene.reactions)
    return [rxn for rxn in rxn_set if not rxn.functional]

### Because the above described code didnt work, we knocked out the PncC gene in the script [NR_pathway](https://github.com/27410/27410-group-assigment-group-1-nmn-production-with-e-coli/blob/main/scripts/NR_pathway.ipynb) and [NA_pathway](https://github.com/27410/27410-group-assigment-group-1-nmn-production-with-e-coli/blob/main/scripts/NA_pathway.ipynb).

## Conlcusion: 
To create three different GSM models, that are similiar to the pathways deschribed by Huang, Zhongshi, et al., it is not necessary to delete all the genes in the GSM, that were knocked out in the lab by Huang, Zhongshi, et. because they were either not present in the GSM or the genes were associated to different reactions, which played no role for the NMN pathways.  

Interprating the zero growth of *e.coli* that results in the only gene knock out, it is to be said, that according to the GSM the host will not have any growth and therefore no imporoved NMN prouction. 

Moreover it must be said, that the above code, which was meant to include the deletions into the GSM is not running due to an unsolvable error.